In [1]:
import pandas as pd
import copy
import json
# 读取轨迹数据
df_traj = pd.read_csv(f'../traj_station_handle/df_0.csv')

# 读取uli_index_dict.json数据
with open(f'/root/autodl-tmp/data/operator0/extra_file/uli_index_dict.json', 'r', encoding='utf-8') as file:
    uli_index_dict = json.load(file)

In [2]:
# 定义映射函数
def map_traj_values(traj_list):
    return [uli_index_dict.get(uli, uli) for uli in eval(traj_list)]

# 应用映射函数并创建新列
df_traj['traj_index'] = df_traj['traj'].apply(map_traj_values)

In [3]:
df_traj.head()

,Unnamed: 0,user,day,trajlen,traj,timestamps,traj_index
0,39,a8341179bb6cc,20241001,32,"['0-4-128089867', '0-4-126402305', '0-4-126409...","[1727748066, 1727748085, 1727748198, 172774827...","[2581, 2212, 2237, 1700, 1694, 883, 1362, 756,..."
1,66,a83411893676c,20241005,31,"['0-4-126079489', '0-4-10693890', '0-4-1259274...","[1728104873, 1728104886, 1728104888, 172810489...","[1359, 480, 888, 2895, 1359, 885, 1695, 1698, ..."
2,112,a834118c36b65,20241003,36,"['0-4-132125662', '0-4-17564055', '0-4-1262261...","[1727920865, 1727920876, 1727920878, 172792129...","[3249, 4450, 1780, 4409, 4703, 5859, 3249, 177..."
3,152,a834118c43a7a,20241006,62,"['0-4-132211698', '0-4-17285513', '0-4-1728550...","[1728202111, 1728202263, 1728202310, 172820231...","[3270, 4198, 4195, 493, 4258, 4172, 422, 318, ..."
4,170,a834118c45c96,20241004,31,"['0-4-202709165', '0-4-202709125', '0-4-174882...","[1728039682, 1728039730, 1728039924, 172804008...","[5050, 5047, 4418, 4421, 643, 648, 3574, 894, ..."


In [15]:
all_transitions = []
prev_end = None
for _, row in df_traj.iterrows():
    traj_seq = row['traj_index']
    if prev_end is not None:
        all_transitions.append((prev_end, traj_seq[0]))
    for i in range(len(traj_seq) - 1):
        transition = (traj_seq[i], traj_seq[i + 1])
        all_transitions.append(transition)
    prev_end = traj_seq[-1]

unique_transitions = set(all_transitions)
unique_transitions_list = list(unique_transitions)

In [16]:
print(unique_transitions_list[:10])

[(586, 3656), (257, 4878), (4882, 1219), (2867, 3719), (4543, 1753), (2232, 2979), (1091, 2825), (31, 2845), (683, 556), (2949, 6003)]


In [17]:
# 写入vgae的txt文件
with open('../vgae/transitions.txt', 'w', encoding='utf-8') as file:
    for trans in unique_transitions_list:
        file.write(f"{trans[0]} {trans[1]}\n")

In [ ]:
# 写入node2vec的txt文件
with open('transitions_weight.txt', 'w', encoding='utf-8') as file:
    for trans in unique_transitions_list:
        file.write(f"{trans[0]} {trans[1]} 1\n")